<a href="https://colab.research.google.com/github/vasumaha/Genai_notes/blob/main/20_LCEL_configurable_fields.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install langchain langchain_openai --upgrade

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "API_KEY_HERE"

Oftentimes you may want to experiment with, or even expose to the end user, multiple different ways of doing things. In order to make this experience as easy as possible, we have defined two methods.

First, a `configurable_fields` method. This lets you configure particular fields of a runnable.

Second, a `configurable_alternatives` method. With this method, you can list out alternatives for any particular runnable that can be set during runtime.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(temperature=0).configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        name="LLM Temperature",
        description="The temperature of the LLM",
    )
)

In [ ]:
model.invoke("pick a random number")

AIMessage(content='17', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 11, 'total_tokens': 12}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-17f337c4-93b6-406d-b78d-9ee2c188b73f-0')

In [ ]:
model.with_config(configurable={"llm_temperature": 0.9}).invoke("pick a random number")

AIMessage(content='27', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 11, 'total_tokens': 12}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-053d586e-f1d0-4d80-9f91-115bbe7e60a4-0')

## Configuring Prompts

In [ ]:
llm = ChatOpenAI(temperature=0)
prompt = PromptTemplate.from_template(
    "Tell me a joke about {topic}"
).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="prompt"),
    # This sets a default_key.
    # If we specify this key, the default LLM (ChatAnthropic initialized above) will be used
    default_key="joke",
    # This adds a new option, with name `poem`
    poem=PromptTemplate.from_template("Write a short poem about {topic}"),
    # You can add more configuration options here
)
chain = prompt | llm

In [ ]:
# By default it will write a joke
chain.invoke({"topic": "bears"})

AIMessage(content="Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship any longer!", response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 13, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-eb2be3f4-df43-437f-8dc7-94053170ab30-0')

In [ ]:
# We can configure it write a poem
chain.with_config(configurable={"prompt": "poem"}).invoke({"topic": "bears"})

AIMessage(content="In the forest deep and wide,\nWhere the shadows dance and hide,\nThere roams a creature strong and grand,\nWith fur as dark as midnight's hand.\n\nThe bear, with paws like mighty claws,\nStands tall and proud, without a pause,\nIn search of food and shelter warm,\nIn the wild, where it was born.\n\nWith a growl that shakes the earth,\nAnd eyes that hold both fear and mirth,\nThe bear is a symbol of strength and might,\nA guardian of the day and night.\n\nSo let us respect this noble beast,\nAnd in its presence, be at peace,\nFor in the wild, where it roams free,\nThe bear is a wonder to see.", response_metadata={'token_usage': {'completion_tokens': 142, 'prompt_tokens': 13, 'total_tokens': 155}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-48a7d22d-047b-4d56-8531-b297c70c87b6-0')

## Saving Configurations

In [ ]:
openai_joke = chain.with_config(configurable={"llm": "openai"})

'{"name": null, "bound": {"name": null, "first": {"name": null, "default": {"name": null, "input_variables": ["topic"], "input_types": {}, "output_parser": null, "partial_variables": {}, "metadata": null, "tags": null, "template": "Tell me a joke about {topic}", "template_format": "f-string", "validate_template": false, "_type": "prompt"}, "config": null, "which": ["prompt", null, null, null, false], "alternatives": {"poem": {"name": null, "input_variables": ["topic"], "input_types": {}, "output_parser": null, "partial_variables": {}, "metadata": null, "tags": null, "template": "Write a short poem about {topic}", "template_format": "f-string", "validate_template": false, "_type": "prompt"}}, "default_key": "joke", "prefix_keys": false}, "middle": [], "last": {"model_name": "gpt-3.5-turbo", "model": "gpt-3.5-turbo", "stream": false, "n": 1, "temperature": 0.0, "_type": "openai-chat"}}, "kwargs": {}, "config": {"configurable": {"llm": "openai"}}, "config_factories": [], "custom_input_typ